In [25]:
# önce ayarları yaptım ve dosyaları okuttum

import lightgbm as gb
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from boruta import BorutaPy
import os

os.getcwd()
os.chdir("C:/Users/Yusuf/Desktop/6_semester_yapay_zeka/proje/")
os.getcwd()

pd.set_option('display.max_columns', 100)
#pd.set_option("display.max_rows", None)
pd.options.mode.copy_on_write = True

# pd.reset_option('display.max_columns')  
#pd.reset_option("display.max_rows")  

# output verileri ile oynama kodları

In [26]:
# burada semih hocanın sonradan attığı dosyadaki kanser hücre hattı adları için 
# CCLe ve GDSC adları eşleştirmesi verisinde hata olup olmadığını kontrol ettim, 
# bu sayede diğer iki traindata verisini düzenlerken buna göre çalışabilirim 

dfxx = pd.read_csv("datalar/CCLE_samples.tsv",sep="\t")

#dfxx, len(dfxx["Accession"].unique()) == len(dfxx["Title"].unique())

# burada da ek dosyadaki title'ları bir listede toparladım ve gen adı anlamına gelen "ENTREZID" ismini ilk index'e ekledim
# bu kısıma alternatif olarak .tolist()'de kullanılabilir, hatta daha mantıklı olur ancak burada onu tam kavrayamamışım

dfxx = pd.read_csv("datalar/CCLE_samples.tsv",sep="\t")
abc = []
for i in dfxx["Title"]:
    abc.append(i)
abc.insert(0, "ENTREZID")
#abc

In [27]:
# burada train edeceğim CCLE expresyon datasını okuttum 
# ve sütun adlarının test dosyasıyla eşleşmesi için
# daha önceki dosyadan aldığım sütun adlarıyla değiştirdim
# ilk sütun exprese edilen gen adı, kalan sütunlar samplelar, 
# yani senin elinde 917 farklı kanser örneği varsa bu o işte

df1 = pd.read_csv("datalar/traindataCCLE.tsv",sep="\t")

df1.columns = abc
columnnamesnew = df1["ENTREZID"].tolist()
traindataTra = df1.T
traindataTra.rename(columns = traindataTra.iloc[0])
traindataTra.drop("ENTREZID", inplace=True)
traindataTra.columns = columnnamesnew
traindataTra

# training için kullanacağımız datamız traindataTra olacak

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADAT,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARS1,AARS2,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,ABCC10,ABCC11,ABCC12,...,ZNRF3,ZNRF4,ZP1,ZP2,ZP4,ZPBP,ZPBP2,ZPLD1,ZPR1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1321N1,5.057414,3.101889,3.868946,3.27403,4.096619,3.657733,6.270943,5.504679,4.300312,3.202268,3.193128,5.963959,7.081995,4.554877,7.684224,9.098974,3.283693,6.270091,10.338294,5.705048,7.151537,9.333823,6.657706,9.632623,5.133462,3.313759,6.360862,3.197461,3.237088,4.193795,6.210366,3.750262,4.796452,3.308814,4.162281,3.880552,3.247615,3.259028,8.569443,3.321863,3.661906,3.276531,7.131393,9.394601,4.430886,4.036965,7.882758,6.218446,3.299537,3.318549,...,6.325263,3.853576,3.69463,3.567255,3.455451,3.405473,3.045822,3.222315,8.31334,6.703535,9.857293,4.671025,7.202999,3.575595,3.429045,3.521215,5.203932,6.29183,4.767197,3.94093,7.770187,4.854016,3.854197,5.507511,5.761879,5.897562,4.326148,5.95143,6.068313,3.124634,5.369337,7.286626,5.552407,3.1515,5.533826,3.97801,6.814621,5.360619,6.171234,7.280945,9.198282,10.265829,10.486985,4.18248,5.922202,3.51335,7.308742,6.591888,5.164463,7.991063
143B,4.685349,3.197981,4.551369,3.196331,3.65766,3.770574,6.333388,5.841778,3.949471,3.27855,3.395564,6.290569,7.888878,3.8484,5.872409,9.216004,3.10001,7.379341,10.07224,7.131607,7.499187,9.768383,5.541531,9.622784,4.104243,3.78898,4.185647,3.10237,3.089803,3.876583,5.813258,3.542329,4.130529,3.159042,3.870579,3.313812,3.291164,4.298507,8.636423,3.131696,4.140988,3.296658,6.173316,9.500221,4.076204,3.860301,7.02172,6.766468,3.502191,3.188463,...,7.078895,3.613977,3.930884,3.708599,3.585659,3.574361,3.086734,3.119161,8.254651,7.110148,9.689012,5.454415,5.565606,3.502545,3.446377,4.170758,5.337166,4.720539,4.57991,4.130102,7.653418,4.436534,3.232822,5.760256,5.644289,6.40974,4.692284,3.594436,6.066943,3.082724,5.66945,6.923072,5.545135,3.050207,4.837949,4.159047,8.328085,5.533689,5.720579,7.361637,9.468018,10.277863,11.314794,4.436572,5.820454,6.279338,6.532655,7.470363,6.128307,8.743529
22Rv1,4.151199,6.790822,4.376062,3.137814,3.323012,3.502085,6.961702,7.551065,4.79439,3.153752,3.133599,5.819146,7.392169,4.211142,5.321824,8.670827,3.319844,6.485884,9.888936,6.119295,7.630296,9.018166,4.503751,9.030793,5.156458,4.513463,3.6561,3.270684,3.072664,4.710253,7.037308,4.061389,5.33602,3.139567,5.913501,3.230477,3.311888,3.183859,9.007984,3.142554,4.301361,3.268737,7.830125,9.101133,4.684316,4.150021,6.031316,5.74132,3.789513,3.343977,...,7.30299,3.802758,3.945148,3.501721,3.315404,3.354842,3.192306,3.248203,7.094469,6.535769,9.636822,5.10568,6.206704,3.267481,4.229475,3.424062,5.967695,4.589245,4.759412,3.643105,7.555819,5.191817,3.532081,5.132131,5.210584,5.566984,5.410131,4.028977,5.853284,3.045039,4.922403,7.232994,5.08404,3.149408,4.387888,5.147897,7.09859,6.417736,6.342124,8.273656,9.122201,9.692324,11.42644,4.360267,5.723774,3.75789,6.586693,4.94054,5.094299,8.16281
23132/87,3.664683,5.446926,3.506436,3.274304,3.895354,3.544555,5.99701,6.348905,3.822763,3.449912,3.592802,3.936781,7.964486,3.895532,6.418828,8.240881,2.929752,6.554548,10.655967,5.841845,7.172219,8.976018,4.309944,9.292505,6.168674,3.74147,3.526996,7.547983,3.168702,4.671211,4.398491,3.511865,5.017361,3.508044,5.857106,3.30394,3.320122,3.383489,8.061736,3.05997,3.876288,3.198179,7.264386,9.52238,4.240572,4.103817,6.802327,6.712355,3.362996,3.29251,...,7.725889,3.489898,3.490377,3.46187,3.392278,3.718737,3.404624,3.15376,7.297353,6.884179,9.151779,5.021071,6.299837,3.495401,3.304433,3.50945,5.828962,3.250869,4.36574,3.815313,6.975329,4.691965,3.155982,

In [28]:
# burada da test verimi okuttum, 
# sütunları tıpkı bir önceki dosyada olduğu gibi alfabetik sıraladım

df2 = pd.read_csv("datalar/testdataGDSC.tsv",sep="\t")
dfs = sorted(df2)
df2 = df2[dfs]

column_ENTREZID = df2.pop('ENTREZID')
df2.insert(0, 'ENTREZID', column_ENTREZID)
df2 = df2.T
df2.rename(columns = df2.iloc[0])
df2.drop("ENTREZID", inplace=True)
df2.columns = columnnamesnew
testdataTra = df2
testdataTra

#len(df2), len(df2["ENTREZID"].unique())

C:\Users\Yusuf\AppData\Local\Temp\ipykernel_13708\1269621714.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2.insert(0, 'ENTREZID', column_ENTREZID)


,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADAT,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARS1,AARS2,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,ABCC10,ABCC11,ABCC12,...,ZNRF3,ZNRF4,ZP1,ZP2,ZP4,ZPBP,ZPBP2,ZPLD1,ZPR1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
201T,5.856445,3.247484,3.579458,3.162129,3.512963,3.510654,6.76933,6.431106,3.895801,3.416377,3.375519,4.966667,7.990054,4.153196,7.333702,8.965516,3.333252,6.119801,8.058831,6.718742,6.916211,9.862415,3.335317,9.345743,5.165807,5.456806,4.077137,3.157282,2.907463,3.78582,7.605454,3.838774,4.769633,3.360732,3.83692,2.942379,3.309855,3.26306,8.032058,3.143029,3.659782,3.192744,7.391875,9.954695,4.244069,3.99628,7.383772,7.821302,3.019244,3.061286,...,5.19224,3.538553,3.557221,3.495325,3.350115,3.119346,3.235976,3.472954,7.562684,7.535238,9.706463,5.037403,6.904293,3.606627,3.533025,3.543483,5.924577,8.272444,6.151929,3.962316,7.622415,4.844344,3.286574,5.261229,6.68323,6.142908,4.808359,5.648153,6.370307,3.249178,5.376503,7.722946,5.267775,3.113104,4.355214,4.999836,7.495168,4.661365,6.684744,8.205236,9.100823,9.737563,11.236739,4.665385,6.213484,7.210581,7.793637,5.359173,5.635009,7.907468
22RV1,4.780793,3.158834,5.476348,3.212497,3.360919,3.462339,5.624908,7.071681,4.606033,3.444969,3.090546,4.386424,7.009976,3.900099,7.064845,7.982684,3.253945,6.799465,9.109873,5.795986,7.033246,9.589005,3.860537,8.85246,6.608232,12.201327,3.50803,3.288688,2.932061,3.886149,7.53942,3.84392,4.832054,3.210592,4.256684,3.024886,3.348865,4.240176,8.840207,2.965257,7.480513,3.085353,6.390294,9.079754,4.118359,4.118416,7.228683,7.29169,3.11121,3.356265,...,7.482202,3.785289,4.060327,3.568538,3.233406,3.56016,3.125234,3.268395,6.037385,6.961676,8.559896,5.099874,5.791882,3.55475,3.609464,4.052643,6.348177,5.824185,4.589431,3.744036,6.447635,4.60469,3.316553,4.423569,6.974051,6.011159,5.869216,6.712298,5.917183,3.146958,4.67021,8.120983,5.456767,3.389625,5.430324,7.688864,7.743179,5.232117,6.438865,7.636993,8.163951,8.41775,10.730596,4.767418,5.918395,4.785197,6.350664,5.277293,5.366602,6.702161
23132-87,5.366871,3.267892,4.085302,3.183918,4.128507,3.565163,5.586122,5.574648,3.826814,7.359631,3.457021,5.569545,6.494175,4.232994,7.399241,8.299986,3.166962,6.51295,8.825367,6.026927,6.807911,7.748977,6.096744,9.277104,4.323229,3.716042,8.39101,3.226284,3.870457,3.638732,3.677445,4.013638,5.082889,3.232078,4.63842,3.686535,3.390117,3.25795,8.390482,3.159021,3.581347,3.24938,6.036672,8.907511,4.286127,4.104107,7.539158,6.016039,3.3737,3.287498,...,6.641014,3.549193,3.874059,3.597081,3.192695,3.509982,3.429012,3.086935,6.354628,6.314284,9.684392,5.779108,6.419159,3.441996,3.388628,3.709449,5.069306,5.934577,4.45919,4.090031,7.956188,4.723742,3.278527,5.174245,6.128455,6.176689,5.019539,5.541767,5.302129,3.314947,4.227695,7.328939,5.64532,3.318815,4.623318,5.977188,7.066167,6.018858,5.766447,6.612018,6.682025,8.562659,10.058435,4.134415,5.633689,3.943279,6.613023,6.48086,4.817533,7.970788
42-MG-BA,7.600886,9.674856,7.526271,3.372987,3.823559,3.77147,7.50602,8.172882,5.944238,3.51167,3.478824,4.663356,7.125216,4.055572,7.380681,8.786595,3.20995,7.036569,11.080695,6.399369,7.525719,8.085666,3.503754,9.099804,5.860496,5.926854,9.657842,3.297467,3.231512,4.237018,6.606453,4.228788,4.107979,3.261052,5.090191,3.20579,3.503672,3.422507,8.961465,3.273485,3.94012,3.168122,7.213464,7.892262,4.640024,4.320353,7.141192,6.386903,3.463131,3.21732,...,5.844631,3.825267,3.705087,3.565888,3.622564,3.333455,3.320446,3.182156,7.67774,6.830765,9.072901,4.898505,6.281972,3.500521,3.531333,3.438782,3.635068,7.156922,5.792335,3.660409,7.556515,5.03015,3

In [29]:
# burada ilk önce ilaç datasını okuttum ilaçların kopyalarını buldum,
# farklı isimde olanları düzenleyip aynı isime getirdim,
# mesela "NVP-TAE684" ilacı burada sadece "TAE684" diye yazılmış
# sonra da ilaç dosyasından yalnızca işime yarar değerleri çektim

df3 = pd.read_csv("datalar/antikanserCCLE.csv",sep=",")
dfc = df3["Compound"].tolist()
list1 = ["Erlotinib", "PHA-665752","Paclitaxel","Sorafenib","NVP-TAE684", "Crizotinib","AZD-0530","Lapatinib","Nilotinib","17-AAG","PLX4720","PD-0332991","PD-0325901","AZD6244", "Nutlin-3a"]

for i in range(len(dfc)):
    if dfc[i] == "TAE684":
        dfc[i] = "NVP-TAE684"
    if dfc[i] == "AZD0530":
        dfc[i] = "AZD-0530"
    if dfc[i] == "PF2341066":
        dfc[i] = "Crizotinib"
    if dfc[i] == "Nutlin-3":
        dfc[i] = "Nutlin-3a"
df3["Compound"] = dfc
filtredf3 = df3[["CCLE Cell Line Name", "Primary Cell Line Name", "Compound", "IC50 (uM)"]]

In [30]:
# şimdi de iki veri setinde de ortak olan ilaçları da dahil ederek bi selection yaptım, 
filttre = filtredf3

totaldrugs = filttre["Compound"].tolist()
yoklar = list(set(totaldrugs) - set(list1))

xd = []
for i in range(len(totaldrugs)):
    if totaldrugs[i] not in yoklar:
        xd.append(i)

filttre = filttre.iloc[xd].reset_index(drop=True)

In [31]:
# bu kodda CCLE names dosyasını okuttum
# bu dosyada kaç tane sütun ve bunlar için
# kaç tane unic eleman var onu görmüş oldum
# aynısını gdsc için yapıp ortak kanser hatlarını seçip
# o şekilde ilerleyeceğim

dfcclenames = pd.read_csv("datalar/isimlerCCLE.txt", sep = "\t")
listtnames = []
özelller = []
özellsayılar = []

for i in range(len(dfcclenames.columns.tolist())):
    listtnames.append(dfcclenames.columns[i])

for i in listtnames:
    özelller.append(i)
    özellsayılar.append(len(list(set(dfcclenames[i].tolist()))))

dffd = {"sütunlar": özelller, "sayılar": özellsayılar}
dffdd = pd.DataFrame(data = dffd)
dffdd

özgünlerr = list(set(list(dfcclenames["Site Primary"])))
süxxun = []

for i in özgünlerr:
    süxxun.append(dfcclenames["Site Primary"].value_counts()[i])

fff3 = {"siteCCLE": özgünlerr, "numbers": süxxun}
dffveelen = pd.DataFrame(data=fff3)
dffveelen2 = dffveelen[dffveelen["numbers"] >= 20].reset_index(drop=True)
dffveelen2#, dffveelen2["numbers"].sum()

,siteCCLE,numbers
0,bone,29
1,stomach,38
2,urinary_tract,28
3,ovary,52
4,breast,60
5,lung,187
6,skin,62
7,soft_tissue,21
8,upper_aerodigestive_tract,33
9,oesophagus,27


In [32]:
banalazım = dfcclenames["Cell line primary name"].tolist()
traindatatrasatırlar = traindataTra.index.tolist()

for i in dfcclenames.index.tolist():
    if banalazım[i] not in traindatatrasatırlar:
        dfcclenames["Cell line primary name"].drop(index=i)

In [33]:
dfgdscnames = pd.read_csv("datalar/isimlerGDSCcsv.csv", sep = ";")
listnames = []
özeller = []
özelsayılar = []

for i in range(len(dfgdscnames.columns.tolist())):
    listnames.append(dfgdscnames.columns[i])

for i in listnames:
    özeller.append(i)
    özelsayılar.append(len(list(set(dfgdscnames[i].tolist()))))

dfd = {"sütunlar": özeller, "sayılar": özelsayılar}
dfdd = pd.DataFrame(data = dfd)
dfdd

özgünler = list(set(list(dfgdscnames["Site"])))
süxun = []

for i in özgünler:
    süxun.append(dfgdscnames["Site"].value_counts()[i])

ff3 = {"siteGDSC": özgünler, "numbers": süxun}
dffvelen = pd.DataFrame(data=ff3)
dffvelen2 = dffvelen[dffvelen["numbers"] >= 20].reset_index(drop=True)
dffvelen2#, dffvelen2["numbers"].sum()

,siteGDSC,numbers
0,bone,44
1,stomach,29
2,urinary_tract,20
3,autonomic_ganglia,34
4,breast,52
5,ovary,45
6,lung,179
7,skin,62
8,upper_aerodigestive_tract,43
9,soft_tissue,23


In [34]:
dfgdscnames["Line"] = sorted(dfgdscnames["Line"])
dfgdscnames

,Line,COSMIC_ID,Site,Histology
0,201T,905944,lung,carcinoma
1,22RV1,908128,skin,malignant_melanoma
2,23132-87,910700,large_intestine,carcinoma
3,42-MG-BA,1290905,breast,carcinoma
4,451Lu,1297439,lung,carcinoma
...,...,...,...,...
1024,YT,753624,pancreas,carcinoma
1025,ZR-75-30,905988,skin,malignant_melanoma
1026,huH-1,1240185,lung,carcinoma
1027,no-10,907000,endometrium,carcinoma


In [35]:
# burada iki veri setinde de ortak bulunan kanser tiplerini düzenledim
# toplam sayıyı görmek için len komutunu kullandım ve toplam 15 ortak kanser
# tipi varmış

sitegdsc = dffvelen2["siteGDSC"].tolist()
siteccle = dffveelen2["siteCCLE"].tolist()
sitegdsc, siteccle
ortaktipler = list(set(siteccle) & set(sitegdsc))

len(ortaktipler), ortaktipler

(15,
 ['bone',
  'lung',
  'stomach',
  'skin',
  'urinary_tract',
  'large_intestine',
  'kidney',
  'breast',
  'ovary',
  'upper_aerodigestive_tract',
  'soft_tissue',
  'oesophagus',
  'pancreas',
  'haematopoietic_and_lymphoid_tissue',
  'central_nervous_system'])

In [36]:
filtrelenmis_dfccle = dfcclenames[dfcclenames['Site Primary'].isin(ortaktipler)].reset_index(drop=True)
filtrelenmis_dfccle = filtrelenmis_dfccle[["Cell line primary name", "Site Primary"]]
#filtrelenmis_dfccle.set_index('Cell line primary name', inplace=True)

filtrelenmis_dfgdsc = dfgdscnames[dfgdscnames["Site"].isin(ortaktipler)].reset_index(drop=True)
filtrelenmis_dfgdsc = filtrelenmis_dfgdsc[["Line", "Site"]] 

In [37]:
filtrelenmis_dfccle


,Cell line primary name,Site Primary
0,1321N1,central_nervous_system
1,143B,bone
2,23132/87,stomach
3,253J-BV,urinary_tract
4,253J,urinary_tract
...,...,...
926,YH-13,central_nervous_system
927,YKG1,central_nervous_system
928,YMB-1,breast
929,ZR-75-1,breast


In [38]:
filtrelenmis_dfgdsc

,Line,Site
0,201T,lung
1,22RV1,skin
2,23132-87,large_intestine
3,42-MG-BA,breast
4,451Lu,lung
...,...,...
882,YMB-1-E,lung
883,YT,pancreas
884,ZR-75-30,skin
885,huH-1,lung


In [39]:
# bu kısımda tüm ccle veritabanı içerisinde bulunan ve tüm veriler için ortak olan hücre hatları seçildi
# bu sayede potansiyel bir veri kaybı önlendi ve minimum ortak veriler toparlandı

hepsinde_ortak_hatlar_ccle = sorted(list(set(traindataTra.T.columns.tolist()) & set(filttre["Primary Cell Line Name"].tolist()) & set(dfcclenames["Cell line primary name"].tolist())))
filtrelenmis_dfccle[filtrelenmis_dfccle["Cell line primary name"].isin(hepsinde_ortak_hatlar_ccle)]
filtrelenmis_dfccle
#traindataTra = traindataTra.loc[sorted(hepsinde_ortak_hatlar_ccle)]
#filttre = filttre[filttre['Primary Cell Line Name'].isin(hepsinde_ortak_hatlar_ccle)]
#dfcclenames = dfcclenames[dfcclenames["Cell line primary name"].isin(hepsinde_ortak_hatlar_ccle)]

#hepsinde_ortak_hatlar_gdsc = sorted(list(set(testdataTra.T.columns.tolist()) & set(filttre["Primary Cell Line Name"].tolist()) & set(dfcclenames["Cell line primary name"].tolist())))


,Cell line primary name,Site Primary
0,1321N1,central_nervous_system
1,143B,bone
2,23132/87,stomach
3,253J-BV,urinary_tract
4,253J,urinary_tract
...,...,...
926,YH-13,central_nervous_system
927,YKG1,central_nervous_system
928,YMB-1,breast
929,ZR-75-1,breast


In [40]:
# daha önceleri threshhold verilerinin olduğu dosyayı manuel olarak hazırladım ancak bana her türlü antikansergdsc gerekeceğinden onu da okutuyorum 

list2 = ['Erlotinib', 'PHA-665752', 'Paclitaxel', 'Sorafenib', 'NVP-TAE684', 'Crizotinib', 'AZD-0530', 'Lapatinib', 'Nilotinib', '17-AAG', 'PLX4720', 'PD-0332991', 'PD-0325901', 'AZD6244', 'Nutlin-3']
columns = []
for i in list2:
    columns.append(i)
columns[-1] = "Nutlin-3a"
columns

df4 = pd.read_csv("datalar/threshholdGDSC.csv", sep = ";")
antikanserGDSC = pd.read_excel("datalar/antikanserGDSC2.xlsx")

thresholdlar = df4[columns].T
compunds = []
#IC50_uMs = thresholdlar.loc["0"].tolist()
#IC50_uMs

thresholdlar.columns = ["thresholds"]
thresholdlar["Compound"] = thresholdlar.index
thresholdlar.reset_index(drop=True, inplace=True)

virgüllüler = thresholdlar["thresholds"].tolist()
for i in range(len(virgüllüler)):
    virgüllüler[i] = virgüllüler[i].replace(',','.')
    virgüllüler[i] = float(virgüllüler[i])

thresholdlar["thresholds"] = virgüllüler
thresholdlar.dtypes, thresholdlar

(thresholds    float64
 Compound       object
 dtype: object,
     thresholds    Compound
 0      1.56710   Erlotinib
 1      1.90640  PHA-665752
 2     -5.67720  Paclitaxel
 3      0.61159   Sorafenib
 4     -1.12560  NVP-TAE684
 5      1.25310  Crizotinib
 6      0.96253    AZD-0530
 7      1.62570   Lapatinib
 8      1.67950   Nilotinib
 9     -3.26790      17-AAG
 10     2.36910     PLX4720
 11    -0.42572  PD-0332991
 12    -2.73310  PD-0325901
 13     1.20620     AZD6244
 14     2.80630   Nutlin-3a)

In [41]:
abcds = antikanserGDSC["Screened Compounds:"].drop(0)
abcds

1           A253
2       BB30-HNC
3       BB49-HNC
4            BHY
5         BICR10
          ...   
997        TMK-1
998         KP-2
999         KO52
1000        SC-1
1001       U-CH2
Name: Screened Compounds:, Length: 1001, dtype: object

In [42]:
# elimizde olan threshhold değerlerini yukarıda temizlediğimiz
# ccle antikanser datasına uyguluyorum

filttre["log2(IC50)"] = np.log2(filttre['IC50 (uM)'])
filttre_hassas = pd.merge(filttre, thresholdlar, on='Compound', how='inner')
filttre_hassas["Res/Sens"] = np.where(filttre_hassas["log2(IC50)"] > filttre_hassas["thresholds"], "R", "S")
filttre_hassas_yeni = filttre_hassas[["Primary Cell Line Name", "Compound", "Res/Sens"]]
filttre_hassas_yeni.set_index("Primary Cell Line Name", inplace=True)
#filttre_hassas_yeni[["Compound", "Res/Sens"]]
filttre_hassas

,CCLE Cell Line Name,Primary Cell Line Name,Compound,IC50 (uM),log2(IC50),thresholds,Res/Sens
0,22RV1_PROSTATE,22Rv1,Nilotinib,8.000000,3.000000,1.6795,R
1,5637_URINARY_TRACT,5637,Nilotinib,7.475355,2.902142,1.6795,R
2,639V_URINARY_TRACT,639-V,Nilotinib,8.000000,3.000000,1.6795,R
3,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,697,Nilotinib,1.910434,0.933901,1.6795,S
4,769P_KIDNEY,769-P,Nilotinib,8.000000,3.000000,1.6795,R
...,...,...,...,...,...,...,...
7387,WM88_SKIN,WM-88,Erlotinib,8.000000,3.000000,1.5671,R
7388,WM983B_SKIN,WM-983B,Erlotinib,8.000000,3.000000,1.5671,R
7389,YKG1_CENTRAL_NERVOUS_SYSTEM,YKG1,Erlotinib,8.000000,3.000000,1.5671,R
7390,ZR751_BREAST,ZR-75-1,Erlotinib,8.000000,3.000000,1.5671,R


In [43]:
filttre_hassas_yenilerr = filttre_hassas_yeni.pivot_table(index = "Primary Cell Line Name", columns="Compound", aggfunc='first')
filttre_hassas_yenilerr = filttre_hassas_yenilerr["Res/Sens"]
filttre_hassas_yenilerr

Compound,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Primary Cell Line Name,,,,,,,,,,,,,,,
1321N1,R,R,R,R,R,R,R,NaN,R,R,NaN,R,R,S,R
22Rv1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
42-MG-BA,S,R,R,R,R,R,R,NaN,R,R,NaN,R,R,S,R
5637,S,R,R,R,S,S,R,R,R,R,R,R,S,S,R
639-V,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM1799,S,R,S,R,R,R,R,R,R,S,R,R,S,R,R
YKG1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
ZR-75-1,R,R,R,R,R,R,R,NaN,S,R,NaN,R,R,S,R


In [44]:
traindataTra

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AACSP1,AADAC,AADACL2,AADAT,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARS1,AARS2,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB1,ABCB10,ABCB11,ABCB4,ABCB5,ABCB6,ABCB7,ABCB8,ABCB9,ABCC1,ABCC10,ABCC11,ABCC12,...,ZNRF3,ZNRF4,ZP1,ZP2,ZP4,ZPBP,ZPBP2,ZPLD1,ZPR1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
1321N1,5.057414,3.101889,3.868946,3.27403,4.096619,3.657733,6.270943,5.504679,4.300312,3.202268,3.193128,5.963959,7.081995,4.554877,7.684224,9.098974,3.283693,6.270091,10.338294,5.705048,7.151537,9.333823,6.657706,9.632623,5.133462,3.313759,6.360862,3.197461,3.237088,4.193795,6.210366,3.750262,4.796452,3.308814,4.162281,3.880552,3.247615,3.259028,8.569443,3.321863,3.661906,3.276531,7.131393,9.394601,4.430886,4.036965,7.882758,6.218446,3.299537,3.318549,...,6.325263,3.853576,3.69463,3.567255,3.455451,3.405473,3.045822,3.222315,8.31334,6.703535,9.857293,4.671025,7.202999,3.575595,3.429045,3.521215,5.203932,6.29183,4.767197,3.94093,7.770187,4.854016,3.854197,5.507511,5.761879,5.897562,4.326148,5.95143,6.068313,3.124634,5.369337,7.286626,5.552407,3.1515,5.533826,3.97801,6.814621,5.360619,6.171234,7.280945,9.198282,10.265829,10.486985,4.18248,5.922202,3.51335,7.308742,6.591888,5.164463,7.991063
143B,4.685349,3.197981,4.551369,3.196331,3.65766,3.770574,6.333388,5.841778,3.949471,3.27855,3.395564,6.290569,7.888878,3.8484,5.872409,9.216004,3.10001,7.379341,10.07224,7.131607,7.499187,9.768383,5.541531,9.622784,4.104243,3.78898,4.185647,3.10237,3.089803,3.876583,5.813258,3.542329,4.130529,3.159042,3.870579,3.313812,3.291164,4.298507,8.636423,3.131696,4.140988,3.296658,6.173316,9.500221,4.076204,3.860301,7.02172,6.766468,3.502191,3.188463,...,7.078895,3.613977,3.930884,3.708599,3.585659,3.574361,3.086734,3.119161,8.254651,7.110148,9.689012,5.454415,5.565606,3.502545,3.446377,4.170758,5.337166,4.720539,4.57991,4.130102,7.653418,4.436534,3.232822,5.760256,5.644289,6.40974,4.692284,3.594436,6.066943,3.082724,5.66945,6.923072,5.545135,3.050207,4.837949,4.159047,8.328085,5.533689,5.720579,7.361637,9.468018,10.277863,11.314794,4.436572,5.820454,6.279338,6.532655,7.470363,6.128307,8.743529
22Rv1,4.151199,6.790822,4.376062,3.137814,3.323012,3.502085,6.961702,7.551065,4.79439,3.153752,3.133599,5.819146,7.392169,4.211142,5.321824,8.670827,3.319844,6.485884,9.888936,6.119295,7.630296,9.018166,4.503751,9.030793,5.156458,4.513463,3.6561,3.270684,3.072664,4.710253,7.037308,4.061389,5.33602,3.139567,5.913501,3.230477,3.311888,3.183859,9.007984,3.142554,4.301361,3.268737,7.830125,9.101133,4.684316,4.150021,6.031316,5.74132,3.789513,3.343977,...,7.30299,3.802758,3.945148,3.501721,3.315404,3.354842,3.192306,3.248203,7.094469,6.535769,9.636822,5.10568,6.206704,3.267481,4.229475,3.424062,5.967695,4.589245,4.759412,3.643105,7.555819,5.191817,3.532081,5.132131,5.210584,5.566984,5.410131,4.028977,5.853284,3.045039,4.922403,7.232994,5.08404,3.149408,4.387888,5.147897,7.09859,6.417736,6.342124,8.273656,9.122201,9.692324,11.42644,4.360267,5.723774,3.75789,6.586693,4.94054,5.094299,8.16281
23132/87,3.664683,5.446926,3.506436,3.274304,3.895354,3.544555,5.99701,6.348905,3.822763,3.449912,3.592802,3.936781,7.964486,3.895532,6.418828,8.240881,2.929752,6.554548,10.655967,5.841845,7.172219,8.976018,4.309944,9.292505,6.168674,3.74147,3.526996,7.547983,3.168702,4.671211,4.398491,3.511865,5.017361,3.508044,5.857106,3.30394,3.320122,3.383489,8.061736,3.05997,3.876288,3.198179,7.264386,9.52238,4.240572,4.103817,6.802327,6.712355,3.362996,3.29251,...,7.725889,3.489898,3.490377,3.46187,3.392278,3.718737,3.404624,3.15376,7.297353,6.884179,9.151779,5.021071,6.299837,3.495401,3.304433,3.50945,5.828962,3.250869,4.36574,3.815313,6.975329,4.691965,3.155982,

In [45]:
# ortak 15 kanser hattına göre hücre hatlarımızın bulunduğu df
print(filtrelenmis_dfccle.shape)

# traindata'mız (CCLE)
print(traindataTra.shape)

# 15 ilacımızın farklı hücre hatları üzerinde etkisi
print(filttre_hassas_yenilerr.shape)

# kanser ve hücre hatları verimizi çekeceğimiz yer de burası filtrelenmis_dfccle2
filtrelenmis_dfccle2 = dfcclenames[dfcclenames['Cell line primary name'].isin(traindataTra.index.tolist())].reset_index(drop=True)
filtrelenmis_dfccle2.shape

#indeksez = traindataTra.index.tolist()#.isin(filtrelenmis_dfccle2["Cell line primary name"]).reset_index(drop=True)
#list(set(indeksez) - set(filtrelenmis_dfccle2["Cell line primary name"].tolist()))
# outputu da şöyle: ['RPMI 6666', 'COLO-699']
traindataTra = traindataTra.drop(['RPMI 6666', 'COLO-699'])

(931, 2)
(917, 17062)
(504, 15)


In [46]:
# şimdi de traindatam ve kanser hatları ve tipleri verimin hücre hatlarının aynı olup olmadığına bakıyorum
# bir önceki hücre de olduğu gibi 915 elemanın hepsi de ortak ve bu istediğim şeydi
len(list(set(traindataTra.index.tolist())&set(filtrelenmis_dfccle2["Cell line primary name"].tolist()))) == 915

True

In [47]:
ilaçlarr = filttre_hassas_yenilerr.columns.tolist()
# ilgileneceğim 15 ortak kanser tipi
kansertipler = filtrelenmis_dfccle["Site Primary"].unique().tolist()

samplelarvetipler = dfcclenames[["Site Primary", "Cell line primary name"]]
samplelarvetipler


,Site Primary,Cell line primary name
0,central_nervous_system,1321N1
1,bone,143B
2,prostate,22Rv1
3,stomach,23132/87
4,urinary_tract,253J-BV
...,...,...
1041,central_nervous_system,YH-13
1042,central_nervous_system,YKG1
1043,breast,YMB-1
1044,breast,ZR-75-1


In [48]:
# burada sıra sıra bütün ilaçlar için y değeri hazır
# sıra x değerini oluşturmakta
for ilaç in ilaçlarr:
    y = filttre_hassas_yenilerr[ilaç]
    print(y)

Primary Cell Line Name
1321N1      R
22Rv1       R
42-MG-BA    S
5637        S
639-V       R
           ..
WM1799      S
YKG1        R
ZR-75-1     R
ZR-75-30    S
huH-1       S
Name: 17-AAG, Length: 504, dtype: object
Primary Cell Line Name
1321N1      R
22Rv1       R
42-MG-BA    R
5637        R
639-V       R
           ..
WM1799      R
YKG1        R
ZR-75-1     R
ZR-75-30    R
huH-1       R
Name: AZD-0530, Length: 504, dtype: object
Primary Cell Line Name
1321N1      R
22Rv1       R
42-MG-BA    R
5637        R
639-V       R
           ..
WM1799      S
YKG1        R
ZR-75-1     R
ZR-75-30    R
huH-1       R
Name: AZD6244, Length: 504, dtype: object
Primary Cell Line Name
1321N1      R
22Rv1       R
42-MG-BA    R
5637        R
639-V       R
           ..
WM1799      R
YKG1        R
ZR-75-1     R
ZR-75-30    R
huH-1       R
Name: Crizotinib, Length: 504, dtype: object
Primary Cell Line Name
1321N1      R
22Rv1       R
42-MG-BA    R
5637        S
639-V       R
           ..
WM1799      R


In [49]:
samplelar = dfcclenames[dfcclenames["Site Primary"] == kansertipler[1]]
samplelar2 = samplelar["Cell line primary name"].tolist()
#traindatatra2 = traindataTra.index.tolist[]
samplelar

,CCLE name,Cell line primary name,Cell line aliases,Gender,Site Primary,Histology,Hist Subtype1,Notes,Source,Expression arrays,SNP arrays,Oncomap,Hybrid Capture Sequencing
1,143B_BONE,143B,NaN,F,bone,osteosarcoma,NS,"Identical lines: HTK-, HOS and 143B share high...",ATCC,MAKER_p_NCLE_RNA7_HG-U133_Plus_2_F09_454702,BOWER_p_NCLE_DNAAffy8_GenomeWideSNP_6_D02_464552,yes,NaN
29,A673_BONE,A-673,NaN,F,bone,Ewings_sarcoma-peripheral_primitive_neuroectod...,NS,NaN,ATCC,CASED_p_NCLE_RNA4_HG-U133_Plus_2_B01_383784,FASTS_p_NCLE_DNAAffy4_S_GenomeWideSNP_6_D12_33...,yes,yes
74,CADOES1_BONE,CADO-ES1,NaN,F,bone,Ewings_sarcoma-peripheral_primitive_neuroectod...,NS,NaN,DSMZ,WATCH_p_NCLE_RNA8_HG-U133_Plus_2_D08_474656,CHARY_p_NCLE_DNAAffy9_GenomeWideSNP_6_E12_490484,yes,yes
86,CAL78_BONE,CAL-78,NaN,M,bone,chondrosarcoma,dedifferentiated,NaN,DSMZ,SILOS_p_NCLE_RNA9_HG-U133_Plus_2_H04_523550,CHARY_p_NCLE_DNAAffy9_GenomeWideSNP_6_G02_490488,yes,yes
194,EW8_BONE,EW8,NaN,NaN,bone,Ewings_sarcoma-peripheral_primitive_neuroectod...,NS,NaN,Academic Lab / Achilles,RILLS_p_AchillesExp_Mar2011_1_HG-U133_Plus_2_D...,EPHOR_p_NCLE_Achilles_DNAAffy15_GenomeWideSNP_...,NaN,NaN
195,EWS502_BONE,EWS502,NaN,NaN,bone,Ewings_sarcoma-peripheral_primitive_neuroectod...,NS,NaN,Academic Lab / Achilles,RILLS_p_AchillesExp_Mar2011_1_HG-U133_Plus_2_A...,EPHOR_p_NCLE_Achilles_DNAAffy15_GenomeWideSNP_...,NaN,NaN
203,G292CLONEA141B1_BONE,"G-292, clone A141B1",NaN,F,bone,osteosarcoma,NS,NaN,ATCC,MAKER_p_NCLE_RNA7_HG-U133_Plus_2_F01_454686,BOWER_p_NCLE_DNAAffy8_GenomeWideSNP_6_C02_464578,yes,NaN
294,HOS_BONE,HOS,NaN,F,bone,osteosarcoma,NS,"Identical lines: HTK-, HOS and 143B share high...",ATCC,AGENT_p_NCLE_RNA6_HG-U133_Plus_2_A08_436610,BOWER_p_NCLE_DNAAffy8_GenomeWideSNP_6_H09_464634,yes,yes
319,HS706T_BONE,Hs 706.T,NaN,F,bone,giant_cell_tumour,NS,NaN,ATCC,AGENT_p_NCLE_RNA6_HG-U133_Plus_2_H08_436438,SKATS_p_NCLE_DNAAffy13_GenomeWideSNP_6_D10_592860,yes,yes
321,HS737T_BONE,Hs 737.T,NaN,F,bone,other,giant_cell_tumour,NaN,ATCC,MAKER_p_NCLE_RNA7_HG-U133_Plus_2_F03_454682,CHARY_p_NCLE_DNAAffy9_GenomeWideSNP_6_A10_490440,yes,yes


In [50]:
filtrelenmis_dfccle2 = filtrelenmis_dfccle2[["Cell line primary name", "Site Primary"]]
filtrelenmis_dfccle2

,Cell line primary name,Site Primary
0,1321N1,central_nervous_system
1,143B,bone
2,22Rv1,prostate
3,23132/87,stomach
4,42-MG-BA,central_nervous_system
...,...,...
910,YH-13,central_nervous_system
911,YKG1,central_nervous_system
912,YMB-1,breast
913,ZR-75-1,breast


bi ol artık be kardeşim kaç saat oldu ya

In [51]:
def prepare_data_for_boruta(drug_name, cancer_type):
    sample_names = filtrelenmis_dfccle2[filtrelenmis_dfccle2['Site Primary'] == cancer_type].index.tolist()
    drug_response_data = filttre_hassas_yenilerr[[drug_name]]
    gene_expression_data_for_cancer_type = traindataTra.iloc[sample_names]
    merged_data = pd.merge(drug_response_data, gene_expression_data_for_cancer_type, left_index=True, right_index=True).dropna()
    X = merged_data.drop(columns=[drug_name])
    y = merged_data[drug_name]
    return X, y

In [52]:
def run_boruta(X, y):
    forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
    boruta_feature_selector = BorutaPy(forest, n_estimators="auto", max_iter=50, verbose=1, random_state=42)
    boruta_feature_selector.fit(X.values, y.values)
    return X.columns[boruta_feature_selector.support_].tolist()

In [53]:
output_file = 'boruta_resultsCCLE.txt'
boruta_results_dict = {}

with open(output_file, 'w') as file:
    for cancer_type in filtrelenmis_dfccle['Site Primary'].unique():
        file.write(f"Cancer Type: {cancer_type}\n")
        boruta_results_dict[cancer_type] = {}
        for drug_name in filttre_hassas_yenilerr.columns:
            file.write(f"Drug: {drug_name}\n")
            X, y = prepare_data_for_boruta(drug_name, cancer_type)
            selected_features = run_boruta(X, y)
            boruta_results_dict[cancer_type][drug_name] = selected_features
            file.write(f"Selected Features: {', '.join(selected_features)}\n\n")

In [55]:
# herhangi bir yapacağım veri dönüşümünde kaybolmaması için kaydettim

borutasonuçlar = boruta_results_dict
dosyasonuçlar = open("dosyasonuçlar1.txt","wt")
dosyasonuçlar.write(str(borutasonuçlar))
dosyasonuçlar.close()

In [56]:
pdpivot = pd.DataFrame(data=borutasonuçlar)
pdpivot = pdpivot.iloc[range(15)]
pdpivot.head()


,central_nervous_system,bone,stomach,urinary_tract,ovary,haematopoietic_and_lymphoid_tissue,kidney,skin,soft_tissue,lung,pancreas,breast,upper_aerodigestive_tract,large_intestine,oesophagus
17-AAG,"[DNAJC17, MICAL3, MRGPRX1, MRPL40, SNAPC1, SNR...",[],"[ABCA2, CPM, DBI, DNAAF9, FEZ2, KCNK10, LINC00...","[ATP7B, HMMR, INPP4B, PCMT1, SLITRK1, TRIM11, ...","[ADORA1, DMRT3, FAM174B, LGR6, LYG2, TUBGCP3, ...","[ARHGAP33, ASXL2, CDKN1A, EFEMP1, PHLDA3, WDR76]",[],"[HECTD2, HTN3, KANSL1L, LSM10, PGAP1, TSR3, YI...","[B3GNT5, C3AR1, CAB39, CAND2, CIITA, GSPT2, H2...","[F2, HS3ST5, NQO1, OGDHL]","[ERCC6, TRAF3IP2, ZNF830]","[ARF6, BCAT2, GSDME, IDI2, SELP]",[],"[CD81, CYB561D1, GTSE1, HERC5, KCNJ2, PABPC1L,...","[DDX3Y, IPCEF1, YBX3]"
AZD-0530,[],[],"[CLPB, CPPED1, DMXL1, MBTD1, RPL23, SETD3]",[],"[CCNG2, CLTCL1, FAM162A, MID1IP1, MIF, PIGS]","[AP2A1, CCDC106, ILRUN, NMRK1, PABPC3, RBPMS2,...",[],[],[],"[AHI1, ALDH1A3, ALPK3, C1GALT1, CLMN, ERMP1, F...","[CALCA, USP22]","[F3, ING5, KCNJ15, LAMP3, MET, NIBAN3, PROSER2]",[],"[CDKN2C, CSF2, HAAO, MCU, PIAS4, SERPINC1, SLC...","[CBX5, COX7A2L, CXCL1]"
AZD6244,[],[],"[CCNG1, DDB2, MDM2, NPAS2, PDE12, RFESD, RPS27...",[],"[EBF2, LRATD2, PAX8, PDGFRA, RPS6KA5, SEC23IP,...","[ADORA2B, APOBR, C12orf60, CAT, CNRIP1, CXCL2,...",[],"[ADAMTS15, AIG1, BCL2A1, CDK5R1, CYGB, DHX33, ...","[CHMP2B, MRPS15]","[H2AC8, H3C4, IDH3A, OCRL, USP3, WDR36]","[CEP104, CXCL10, KRT81]",[],[],"[B3GNTL1, CABYR, CDKL2, HERC5, PRSS50, RRN3]","[ACTR3C, KLF4, MYL11, OLIG3, SCT]"
Crizotinib,[],[],"[AGXT, CLASP2, CYTH1, HAAO, HMGA2, MSC, RGS8, ...",[],[],"[ACOT13, C6orf89, JAKMIP2, PEX13, SH2D4B, SPHK...",[],[],[],"[PSTPIP2, SLAMF6]",[AKAP14],[],[],[],[]
Erlotinib,[],[],"[COMMD6, DPYS, GMFB, KMT2B, SMIM3, SSMEM1, TTL]","[BCAR3, BMAL1, KNOP1, NOD2, PCDH1, SLC35F3, SR...",[],"[AP3M1, MXRA8]",[],[],[],"[ADGRA2, ADGRF4, ANO1, ANO9, ANXA2, CAPN1, CCR...",[],"[CACNG6, CLEC4E]",[],"[ADGRL1, AGO3, INTS15]","[C9orf50, CRMP1, GATA3, GDE1, ITPRIPL2, MAP1B,..."


In [57]:
satırlar = []
for kansertip, ilaçisim in pdpivot.items():
    for ilacisim, features in ilaçisim.items():
        for feature in features:
            satırlar.append([kansertip, ilacisim, feature])

ccledataframe_features = pd.DataFrame(satırlar, columns=["Cancer Type", "Drug Name", "Gene Name"])
ccledataframe_features, len(ccledataframe_features["Gene Name"].unique().tolist())

(                Cancer Type   Drug Name Gene Name
 0    central_nervous_system      17-AAG   DNAJC17
 1    central_nervous_system      17-AAG    MICAL3
 2    central_nervous_system      17-AAG   MRGPRX1
 3    central_nervous_system      17-AAG    MRPL40
 4    central_nervous_system      17-AAG    SNAPC1
 ..                      ...         ...       ...
 800              oesophagus  PD-0325901    ACTR3C
 801              oesophagus  PD-0325901      KLF4
 802              oesophagus  PD-0325901     MYL11
 803              oesophagus  PD-0325901     OLIG3
 804              oesophagus  PD-0325901       SCT
 
 [805 rows x 3 columns],
 716)

In [58]:
len(ccledataframe_features["Cancer Type"].unique().tolist()) # kaç tane kanser tipi için bulduğunu öğreniyorum

13

In [59]:
# burada da hangi kanser tipi için kaç adet gen var ona bakıyorum

satırs = ccledataframe_features["Cancer Type"].unique().tolist()
sütunss = []
for kansertipi in ccledataframe_features["Cancer Type"].unique():
    sütunss.append(ccledataframe_features["Cancer Type"].value_counts()[kansertipi])

dictf = {"kansertipi": satırs, "unic genler":  sütunss}
pdfeatures = pd.DataFrame(dictf)
pdfeatures2 = pdfeatures[pdfeatures["unic genler"] >= 20].reset_index(drop=True)
pdfeatures2

,kansertipi,unic genler
0,stomach,85
1,urinary_tract,20
2,ovary,33
3,haematopoietic_and_lymphoid_tissue,192
4,skin,126
5,soft_tissue,20
6,lung,164
7,pancreas,32
8,breast,38
9,large_intestine,47


In [60]:
filtrelenmis_dfgdsc

,Line,Site
0,201T,lung
1,22RV1,skin
2,23132-87,large_intestine
3,42-MG-BA,breast
4,451Lu,lung
...,...,...
882,YMB-1-E,lung
883,YT,pancreas
884,ZR-75-30,skin
885,huH-1,lung


In [61]:
unic_genler = ccledataframe_features["Gene Name"].unique().tolist()
unic_genler

['DNAJC17',
 'MICAL3',
 'MRGPRX1',
 'MRPL40',
 'SNAPC1',
 'SNRNP40',
 'ATP1A1',
 'CNTN3',
 'DDX20',
 'ABCA2',
 'CPM',
 'DBI',
 'DNAAF9',
 'FEZ2',
 'KCNK10',
 'LINC00922',
 'NQO1',
 'RBM18',
 'RBP3',
 'SIRT4',
 'TFDP1',
 'CLPB',
 'CPPED1',
 'DMXL1',
 'MBTD1',
 'RPL23',
 'SETD3',
 'CCNG1',
 'DDB2',
 'MDM2',
 'NPAS2',
 'PDE12',
 'RFESD',
 'RPS27L',
 'TOB1',
 'AGXT',
 'CLASP2',
 'CYTH1',
 'HAAO',
 'HMGA2',
 'MSC',
 'RGS8',
 'SEBOX',
 'SLC4A7',
 'UBP1',
 'COMMD6',
 'DPYS',
 'GMFB',
 'KMT2B',
 'SMIM3',
 'SSMEM1',
 'TTL',
 'RSRC1',
 'FAM185A',
 'FGF8',
 'LIMA1',
 'PCMTD1',
 'RIC8B',
 'SESN1',
 'SLC25A21',
 'ZNF239',
 'ANKRD24',
 'C2CD6',
 'DENND4C',
 'EBLN2',
 'INPP1',
 'IRS4',
 'KRT24',
 'ZFP69B',
 'ZNF577',
 'BMP2K',
 'CCDC191',
 'CFAP276',
 'MECOM',
 'RANBP1',
 'SHC1',
 'ABCB8',
 'ATP13A1',
 'CBFB',
 'CDV3',
 'CLDN15',
 'CRIM1',
 'CRYL1',
 'DUSP14',
 'GABARAPL1',
 'NBEAL2',
 'OLR1',
 'PAF1',
 'PCGF3',
 'RMI1',
 'RPUSD4',
 'STK38',
 'TMEM52B',
 'TPM4',
 'ATP7B',
 'HMMR',
 'INPP4B',
 'PCMT1'

In [62]:
traindatatracol = traindataTra.columns.tolist()
traindataTranew = traindataTra.T.loc[unic_genler].T
traindataTranew.columns = sorted(traindataTranew.columns)
traindataTranew

,AAMDC,ABCA12,ABCA2,ABCB1,ABCB8,ABCD3,ABRA,ACOT13,ACTR3C,ACYP1,ADAMTS15,ADAMTS3,ADAT2,ADCY10,ADGRA2,ADGRF4,ADGRL1,ADORA1,ADORA2B,AGO3,AGPAT2,AGXT,AHI1,AIG1,AKAP14,AKT1S1,ALDH1A3,ALPK3,AMMECR1L,ANGPTL3,ANKRD24,ANKRD33,ANKRD37,ANKS6,ANO1,ANO7,ANO9,ANXA2,AP1M1,AP2A1,AP3M1,APOBR,ARF6,ARHGAP28,ARHGAP33,ARHGAP9,ASAP2,ASB6,ASXL2,ATF7IP,...,TYW1,UBASH3B,UBE2E2,UBP1,UBTF,UCHL5,UCP2,ULK4,UPP2,USP22,USP27X,USP3,USP43,VAMP8,VAV1,VDR,VLDLR,VPS39,VSIG1,VSIR,WDR36,WDR76,WNT16,WT1,YAF2,YBX3,YIPF1,YWHAG,ZBTB5,ZC3H10,ZEB1,ZFP36,ZFP69B,ZFP92,ZNF114,ZNF155,ZNF239,ZNF285,ZNF333,ZNF347,ZNF488,ZNF575,ZNF577,ZNF599,ZNF629,ZNF630,ZNF750,ZNF75D,ZNF830,ZNRF3
1321N1,5.507849,4.253401,3.255745,9.717647,7.500895,8.134789,9.736803,3.232703,8.178248,4.193795,3.356098,11.691866,5.263362,9.736924,3.270354,3.514263,10.394738,7.562499,4.157663,4.116923,9.38594,5.461242,6.800297,7.49139,5.523728,12.504571,5.724956,9.11736,8.89955,4.544695,4.769397,6.949633,6.223288,9.933551,9.033696,3.745365,6.275275,4.762346,4.00444,5.729082,3.32319,3.388112,3.745544,7.836784,8.859115,7.60945,3.341526,9.158479,4.870038,7.758973,...,6.13705,6.770629,7.09033,8.142496,6.173749,5.080536,7.833173,3.425918,7.786674,6.15415,7.002598,5.057819,5.556194,8.777894,7.688569,3.34076,3.650677,3.418957,4.691378,7.890017,3.346617,3.421388,11.2446,8.237339,11.034805,3.514553,3.621415,7.984956,4.159456,3.283089,3.282546,3.776866,6.115979,3.407324,8.499428,7.472217,9.482038,3.442926,6.740346,8.108366,8.352822,10.207808,8.34472,8.035335,4.444826,3.649121,3.303886,7.329318,11.168509,7.324204
143B,6.554255,5.064579,3.552604,10.195,8.945421,8.742489,10.943282,3.111375,9.550228,3.876583,3.330189,10.038374,5.310234,11.418446,3.660005,3.090326,9.233657,7.792694,4.188328,3.808135,8.778242,5.201025,5.606128,5.814647,5.940625,12.680702,6.976362,10.069993,6.105052,4.330322,5.666259,7.497349,6.025105,9.369276,6.623576,3.819702,6.52722,4.306998,4.325235,4.424589,3.991361,3.14797,3.785721,7.36596,8.875206,7.295795,3.277691,8.938965,4.795111,5.655211,...,6.113596,6.514989,8.047799,7.476958,6.058152,5.440488,8.458072,4.855469,7.486486,4.918925,6.466959,8.340299,5.460491,8.355069,7.552699,3.475705,3.441868,3.194949,5.654715,7.270976,3.246753,3.529933,11.999303,7.439776,11.714428,5.940265,3.786034,7.535542,4.205416,3.128514,3.308515,3.738378,3.823091,4.674411,8.847072,7.627843,9.312824,3.179396,7.188082,7.753739,7.772031,11.696233,8.232147,7.980822,5.598376,3.399483,3.86688,7.409082,10.350896,7.79061
22Rv1,5.576287,4.370801,3.281119,10.142956,6.375373,8.168681,10.179767,5.477527,8.135965,4.710253,3.409483,11.528847,3.986016,8.652732,3.278677,3.318722,8.24142,8.140508,3.976932,4.504054,8.475168,5.076663,5.935249,7.268129,5.950771,12.622599,6.168602,10.271989,7.354374,5.112092,3.801406,7.000463,3.844423,9.518156,11.17242,3.736536,5.825571,4.706254,4.131631,3.725148,3.518302,3.334637,3.924516,5.878669,8.094753,7.955931,3.341522,8.6974,4.638909,2.99745,...,6.511143,7.506892,6.684958,6.933399,5.525641,5.812194,9.355198,5.764655,7.729181,4.09823,3.943286,3.30843,6.094118,6.019832,7.59597,3.568836,3.455186,3.409055,3.409058,4.499434,7.445872,3.36444,12.039446,8.228485,9.709222,3.757626,5.64078,6.675633,4.275866,3.316678,3.44172,3.409028,3.768382,7.025916,9.633263,5.065097,7.187744,3.572854,5.677166,5.257028,6.908537,10.729805,7.259216,5.929009,6.763239,3.683604,5.206401,6.704488,10.277541,7.565149
23132/87,6.153983,4.62385,3.649524,9.579485,5.775534,8.919947,11.073921,3.098333,6.070154,4.671211,3.54552,11.229991,3.576558,9.254945,3.423552,3.117747,13.631544,8.222957,3.438726,3.80652,8.906798,5.463093,5.731843,7.013016,5.743955,12.341372,7.038018,8.441504,9.028593,6.536122,5.422036,6.436713,4.439372,10.557356,9.65418,3.568573,6.665076,5.286072,3.906491,3.337566,3.384622,3.276105,3.833172,6.253239,8.453966,7.790842,3.307677,8.47476,4.486572,3.670103,...,4.447326,6.168723,6.720807,6.623992,5.626999,6.539979,8.555675,7.281196,6.864906,5.072873,8.360657,3.592389,3.940041,3.525965,7.838534,3.973669,3.320992,3.440157,3.583479,6.5

In [63]:
testdatatra2 = testdataTra
testdatatra2 = testdatatra2.T.loc[unic_genler].T
testdatatra2.columns = sorted(testdatatra2.columns)
testdatatra2

,AAMDC,ABCA12,ABCA2,ABCB1,ABCB8,ABCD3,ABRA,ACOT13,ACTR3C,ACYP1,ADAMTS15,ADAMTS3,ADAT2,ADCY10,ADGRA2,ADGRF4,ADGRL1,ADORA1,ADORA2B,AGO3,AGPAT2,AGXT,AHI1,AIG1,AKAP14,AKT1S1,ALDH1A3,ALPK3,AMMECR1L,ANGPTL3,ANKRD24,ANKRD33,ANKRD37,ANKS6,ANO1,ANO7,ANO9,ANXA2,AP1M1,AP2A1,AP3M1,APOBR,ARF6,ARHGAP28,ARHGAP33,ARHGAP9,ASAP2,ASB6,ASXL2,ATF7IP,...,TYW1,UBASH3B,UBE2E2,UBP1,UBTF,UCHL5,UCP2,ULK4,UPP2,USP22,USP27X,USP3,USP43,VAMP8,VAV1,VDR,VLDLR,VPS39,VSIG1,VSIR,WDR36,WDR76,WNT16,WT1,YAF2,YBX3,YIPF1,YWHAG,ZBTB5,ZC3H10,ZEB1,ZFP36,ZFP69B,ZFP92,ZNF114,ZNF155,ZNF239,ZNF285,ZNF333,ZNF347,ZNF488,ZNF575,ZNF577,ZNF599,ZNF629,ZNF630,ZNF750,ZNF75D,ZNF830,ZNRF3
201T,7.065479,4.596568,3.141204,9.647006,6.333973,9.684739,8.072589,3.11045,8.562083,3.78582,3.461116,9.892512,6.558534,9.281794,3.355962,3.339686,8.329786,7.520392,3.557129,5.634019,7.932591,4.78835,4.95226,6.980103,6.572954,12.681074,6.557167,10.415945,7.083723,4.816243,3.870166,6.581181,5.102728,9.871686,9.057282,3.698725,8.011475,4.845793,3.870213,5.365149,3.757108,3.161035,3.667434,7.161646,8.563203,7.649696,4.27775,8.973794,5.264474,4.467009,...,7.122272,6.978208,7.717525,8.402517,5.568292,5.994695,8.865844,7.762419,7.8141,4.654458,6.417934,4.166719,6.45891,6.838897,8.428262,3.324793,3.253452,3.329009,3.697508,6.26054,4.184231,3.460093,10.576049,7.521606,11.524911,3.904755,3.979335,3.626566,3.941269,3.19649,3.281515,3.701157,9.006771,3.45707,9.291257,6.930992,9.743205,3.644447,3.888967,6.835313,7.092683,10.597897,8.522335,3.983728,4.819796,3.2832,3.495498,6.680735,12.67167,7.667526
22RV1,6.163631,4.378325,3.370938,10.147067,6.370541,6.875992,9.531593,3.567423,6.27444,3.886149,4.79035,10.539765,5.300707,8.993698,3.450824,3.188967,8.915714,5.844503,3.885039,3.674069,7.996077,5.109883,5.622434,5.983901,6.567293,12.287764,7.149678,10.196263,7.695377,4.603036,4.650807,6.3135,6.559455,10.095878,9.342755,3.443553,6.464663,5.606467,4.160198,4.630598,3.519691,3.164916,3.820218,5.201514,7.056058,7.584931,3.489522,8.951188,4.512455,4.171835,...,7.817064,6.13342,7.64467,7.473424,5.340832,4.706281,10.045519,5.388822,6.701427,4.350114,9.01713,3.453488,4.089452,9.029781,8.185004,3.294323,3.299468,3.405205,4.798729,6.695578,3.23804,4.078599,3.97881,8.552696,11.533658,3.805416,6.344266,3.813898,3.920759,3.447242,3.276093,3.504496,8.11846,3.415869,9.400094,6.298003,10.435173,5.180812,3.725845,8.049822,5.942861,10.712049,9.181197,3.603587,8.050307,3.351584,6.50157,6.029152,9.918788,7.317484
23132-87,5.939852,4.358547,3.055892,10.316561,7.30968,7.961937,11.160459,3.225417,7.724841,3.638732,3.653515,10.456358,6.866805,10.298768,3.344675,3.344078,9.507623,8.738291,3.686324,3.858712,7.559949,4.76086,4.433474,6.739122,5.969857,12.638368,5.994188,9.255708,9.313356,4.78608,5.384111,5.667629,6.167739,9.036887,10.555131,3.51162,5.557744,5.387546,4.072784,5.980269,3.516429,3.018287,3.955842,7.674242,8.037257,7.970659,3.704692,8.317095,4.260222,5.198704,...,4.935943,6.835077,6.189111,7.401909,5.345256,6.190619,6.949748,4.772184,7.791382,5.362094,7.145672,4.484821,4.00696,8.071064,7.61606,3.285379,3.288752,3.272907,3.941181,6.521791,3.348203,3.634077,12.503254,7.237717,10.635114,4.097196,3.162329,4.234996,4.113594,3.126705,3.356957,3.36406,4.174663,6.006664,9.308595,8.438426,9.807808,3.410399,8.412058,8.374888,6.85221,10.191633,8.468408,9.897045,5.86508,3.582516,10.552822,6.298664,10.814871,7.205803
42-MG-BA,5.876636,4.749692,3.585052,9.887619,5.649629,8.245414,11.27857,3.296506,7.61466,4.237018,3.790034,12.042106,6.091666,9.021502,3.442753,3.250224,12.35481,7.001772,3.607572,4.763208,9.348189,5.242588,6.124448,5.192417,5.990997,12.509725,6.377512,9.65913,6.523027,4.397981,5.499252,6.031228,4.968958,10.740524,10.615781,7.880681,5.882321,4.685179,4.583724,4.056151,3.616452,3.446506,4.138621,5.534202,8.599676,7.832499,3.231096,8.018481,5.023671,4.729042,...,6.908573,4.793352,7.222152,8.781425,5.313263,6.660291,7.77934,6.493171,7.135786,4.754397,4.625237,3.741678,3.697268,7.008227,8.059337,3.578998,3.345722,3.431262,4.382206,5

In [64]:
ilactrainccle = filttre_hassas_yenilerr
trainilaclar = ilactrainccle.index.tolist()
ilactrainccle

Compound,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Primary Cell Line Name,,,,,,,,,,,,,,,
1321N1,R,R,R,R,R,R,R,NaN,R,R,NaN,R,R,S,R
22Rv1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
42-MG-BA,S,R,R,R,R,R,R,NaN,R,R,NaN,R,R,S,R
5637,S,R,R,R,S,S,R,R,R,R,R,R,S,S,R
639-V,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM1799,S,R,S,R,R,R,R,R,R,S,R,R,S,R,R
YKG1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
ZR-75-1,R,R,R,R,R,R,R,NaN,S,R,NaN,R,R,S,R


In [65]:
ilactestgdsc = antikanserGDSC
ilactestgdsc.reset_index(drop=True)
ilactestgdsc = ilactestgdsc.drop(0)
ilactestgdsc.set_index("Screened Compounds:", inplace=True)
ilactestgdsc = ilactestgdsc
ilactestgdscson = ilactestgdsc[ilactrainccle.columns.tolist()]
ilactestgdscson

,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Screened Compounds:,,,,,,,,,,,,,,,
A253,R,R,R,R,S,S,R,R,R,R,R,R,R,R,R
BB30-HNC,R,S,R,R,S,S,R,R,S,S,S,R,R,S,R
BB49-HNC,R,R,S,R,R,R,R,R,S,R,R,R,R,R,R
BHY,R,NaN,S,NaN,NaN,NaN,NaN,NaN,R,R,R,NaN,R,NaN,NaN
BICR10,R,NaN,R,NaN,NaN,NaN,NaN,R,R,R,R,NaN,R,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TMK-1,R,NaN,R,NaN,NaN,NaN,NaN,R,R,S,R,NaN,R,NaN,NaN
KP-2,S,NaN,S,NaN,NaN,NaN,NaN,R,R,S,R,NaN,R,NaN,NaN
KO52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
ilaçlarortakhatlar = list(set(ilactestgdscson.index.tolist())&set(ilactrainccle.index.tolist()))
ilactrainccleson = ilactrainccle
ilactrainccleson.columns = sorted(ilactrainccleson.columns.tolist())

for index, row in ilactrainccleson.iterrows():
    count_R = row.value_counts().get("R", 0)
    count_S = row.value_counts().get("S", 0)
    if count_R > count_S:
        ilactrainccleson.loc[index] = row.fillna("R")
    else:
        ilactrainccleson.loc[index] = row.fillna("S")

ilactrainccleson

,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Primary Cell Line Name,,,,,,,,,,,,,,,
1321N1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
22Rv1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
42-MG-BA,S,R,R,R,R,R,R,R,R,R,R,R,R,S,R
5637,S,R,R,R,S,S,R,R,R,R,R,R,S,S,R
639-V,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM1799,S,R,S,R,R,R,R,R,R,S,R,R,S,R,R
YKG1,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
ZR-75-1,R,R,R,R,R,R,R,R,S,R,R,R,R,S,R


In [67]:
ilactestgdscson = ilactestgdscson.loc[sorted(ilaçlarortakhatlar)]
ilactestgdscson.columns = sorted(ilactestgdscson.columns.tolist())

for index, row in ilactestgdscson.iterrows():
    count_R = row.value_counts().get("R", 0)
    count_S = row.value_counts().get("S", 0)
    if count_R > count_S:
        ilactestgdscson.loc[index] = row.fillna("R")
    else:
        ilactestgdscson.loc[index] = row.fillna("S")

ilactestgdscson

,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Screened Compounds:,,,,,,,,,,,,,,,
42-MG-BA,R,R,R,R,R,R,R,S,R,R,R,R,R,R,R
639-V,R,R,R,R,R,R,R,R,S,S,R,R,R,R,R
769-P,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S
8-MG-BA,R,R,R,R,R,R,R,R,R,R,R,R,S,R,R
8305C,R,R,S,R,R,R,R,R,R,S,R,R,S,R,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VMRC-RCW,R,R,R,R,R,R,R,R,R,R,R,R,R,R,R
VMRC-RCZ,R,R,R,R,R,R,R,R,R,R,R,R,R,R,R
WM-115,R,S,S,S,S,S,S,R,S,S,R,S,S,S,S


In [68]:
# ilaç isimlerinin aynılığını kontrol ediyorum
ilactestgdscson.columns == ilactrainccleson.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [69]:
ilactrainccleson = ilactrainccleson.reindex(ilactestgdscson.index)

In [70]:
ilactrainccleson = ilactrainccleson.dropna()

In [71]:
ilactrainccleson = ilactrainccleson.astype(str)
ilactrainccleson

,17-AAG,AZD-0530,AZD6244,Crizotinib,Erlotinib,Lapatinib,NVP-TAE684,Nilotinib,Nutlin-3a,PD-0325901,PD-0332991,PHA-665752,PLX4720,Paclitaxel,Sorafenib
Screened Compounds:,,,,,,,,,,,,,,,
42-MG-BA,S,R,R,R,R,R,R,R,R,R,R,R,R,S,R
639-V,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
769-P,R,S,R,R,S,S,R,R,S,R,R,R,R,R,R
8-MG-BA,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
8305C,R,R,R,R,R,R,R,R,R,R,R,R,R,S,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VMRC-RCW,R,R,R,S,R,R,R,R,R,R,R,S,R,R,R
VMRC-RCZ,R,R,R,R,R,R,R,R,R,R,R,R,R,R,R
WM-115,S,R,R,R,R,R,R,R,R,R,R,R,R,R,R


In [72]:
testdatatra2 = testdatatra2.reindex(traindataTranew.index)

In [73]:
testdatatra2.dropna()

,AAMDC,ABCA12,ABCA2,ABCB1,ABCB8,ABCD3,ABRA,ACOT13,ACTR3C,ACYP1,ADAMTS15,ADAMTS3,ADAT2,ADCY10,ADGRA2,ADGRF4,ADGRL1,ADORA1,ADORA2B,AGO3,AGPAT2,AGXT,AHI1,AIG1,AKAP14,AKT1S1,ALDH1A3,ALPK3,AMMECR1L,ANGPTL3,ANKRD24,ANKRD33,ANKRD37,ANKS6,ANO1,ANO7,ANO9,ANXA2,AP1M1,AP2A1,AP3M1,APOBR,ARF6,ARHGAP28,ARHGAP33,ARHGAP9,ASAP2,ASB6,ASXL2,ATF7IP,...,TYW1,UBASH3B,UBE2E2,UBP1,UBTF,UCHL5,UCP2,ULK4,UPP2,USP22,USP27X,USP3,USP43,VAMP8,VAV1,VDR,VLDLR,VPS39,VSIG1,VSIR,WDR36,WDR76,WNT16,WT1,YAF2,YBX3,YIPF1,YWHAG,ZBTB5,ZC3H10,ZEB1,ZFP36,ZFP69B,ZFP92,ZNF114,ZNF155,ZNF239,ZNF285,ZNF333,ZNF347,ZNF488,ZNF575,ZNF577,ZNF599,ZNF629,ZNF630,ZNF750,ZNF75D,ZNF830,ZNRF3
42-MG-BA,5.876636,4.749692,3.585052,9.887619,5.649629,8.245414,11.27857,3.296506,7.61466,4.237018,3.790034,12.042106,6.091666,9.021502,3.442753,3.250224,12.35481,7.001772,3.607572,4.763208,9.348189,5.242588,6.124448,5.192417,5.990997,12.509725,6.377512,9.65913,6.523027,4.397981,5.499252,6.031228,4.968958,10.740524,10.615781,7.880681,5.882321,4.685179,4.583724,4.056151,3.616452,3.446506,4.138621,5.534202,8.599676,7.832499,3.231096,8.018481,5.023671,4.729042,...,6.908573,4.793352,7.222152,8.781425,5.313263,6.660291,7.77934,6.493171,7.135786,4.754397,4.625237,3.741678,3.697268,7.008227,8.059337,3.578998,3.345722,3.431262,4.382206,5.15686,3.359457,3.573106,10.002502,7.638108,10.648061,6.356112,4.701426,4.168263,4.406794,3.572453,3.554999,3.534708,4.164796,3.414557,8.263978,6.739871,4.283557,3.584406,4.013437,5.97622,6.803112,11.182614,9.530529,9.004132,5.195873,3.562895,3.506147,7.146928,8.644257,7.970581
5637,7.26119,4.454845,3.545968,10.793172,6.43635,9.021609,9.477026,3.495529,6.642495,4.098115,3.543021,11.537862,3.641629,10.343099,3.520866,3.162341,11.469759,7.377954,3.724864,3.776042,9.273431,4.797993,4.123417,6.721287,6.061969,12.232112,6.4038,10.239736,8.609417,4.81988,4.645706,6.065048,6.666327,9.370586,9.192144,3.521761,6.341874,4.811191,4.280364,4.73336,3.640967,3.230546,3.711781,6.968833,8.876508,7.779571,3.32199,8.302033,5.163629,4.679306,...,5.136952,6.369135,7.579465,7.002658,5.439828,5.230233,8.304512,5.282224,7.495558,4.664973,6.894442,5.892402,3.76893,8.291319,7.87493,3.719315,3.574695,3.417828,3.87072,6.001939,3.310117,3.64945,12.182084,7.097774,10.596381,3.753524,5.007701,4.372971,4.069508,3.086706,3.070815,3.5649,4.052228,9.552739,10.228522,8.277282,4.41013,3.474645,4.48497,10.011069,6.66783,11.001822,8.014834,7.184494,5.705738,3.315651,4.729681,5.920757,11.112804,6.853116
639-V,7.34532,5.132278,3.657288,11.014048,6.439197,8.136032,10.260662,3.678329,7.147418,4.076244,4.066737,11.823663,4.552182,8.554165,3.554297,3.402184,12.387406,8.659932,3.737152,4.46624,8.060659,5.063759,6.627405,6.984507,5.738566,12.491564,7.634301,9.580642,5.004421,4.537275,3.958614,5.98288,5.468075,7.7003,10.801894,3.468914,5.887853,5.047754,4.130952,3.874035,3.929448,3.577478,4.022025,5.331933,8.170821,7.951894,3.506728,7.78109,4.832461,4.527283,...,5.476187,6.038525,6.380762,6.080037,5.564048,5.941238,8.437995,7.143027,7.183938,6.085307,6.885856,3.967008,4.090053,6.009054,6.619795,3.889527,3.49191,3.728276,4.003686,5.971544,3.469865,3.798208,11.352242,7.659263,10.491556,4.17886,4.634508,5.26412,4.331996,3.763875,3.626127,3.512494,4.169345,3.39595,9.468913,8.760185,4.315719,3.615004,4.452292,7.007532,5.960729,11.446252,9.499197,8.267253,5.241495,3.69058,3.522503,6.124346,8.031372,6.92388
647-V,6.312087,4.342596,3.39906,9.460965,7.204289,9.806728,9.860651,3.370769,8.171204,4.016757,7.533048,10.853161,5.241694,11.899575,3.375107,3.250431,11.899216,7.784384,3.841785,4.479327,8.619204,5.464452,3.906652,6.730129,5.654422,12.271716,6.528182,9.287685,7.235712,4.581606,4.066586,6.792451,5.45691,9.551053,7.559579,3.588484,6.030975,4.591374,4.077021,5.488501,3.401209,3.360186,3.793792,8.47798,8.213117,7.544518,3.289874,7.976126,4.780446,4.888527,...,4.920431,6.834765,7.945761,8.03165,5.915799,5.217592,8.008211,5.402054,7.138619,5.371624,5.087785,7.93168,4.387446,6.950453,7.866989,4.098876,3.566657,3.59588,4.847153,6.10

In [74]:
print(ilactrainccleson.shape, ilactestgdscson.shape, traindataTranew.shape, testdatatra2.shape)

(315, 15) (315, 15) (915, 716) (915, 716)


işte artık makine öğrenmesine hazırım! LİGHTGBM

önce ilaç datası için, sonra da expression datası için yapacağım. Şimdi ilaç datası

In [75]:
# burada karşılaştığım asıl sorun ilk önce boş satırların doldurulmasıydı, ben de yoğunluğa göre doldurmayı seçtim
# yani eğer o sütundaki R fazlaysa R ile, S fazlaya S ile dolduruluyor

# bir diğer sorunu ise açıkçası anlayamadım

from sklearn.model_selection import train_test_split, GridSearchCV

X, y = ilactrainccleson, ilactestgdscson
X.shape, y.shape

lgbmmm = gb.LGBMClassifier(n_jobs=-1, class_weight='balanced', max_depth=5, ignore_nan=True)

# önce sütun sütun verilerimi dönüştürüp daha sonrasında lightgbm uyguluyorum

for ilaç in y.columns:
    yilaç = y[ilaç]
    counts = yilaç.value_counts()

    # önce düzgün bir şekilde düzenleyip düzenlemediğini kontrol ettim
    #print(yilaç.tolist())

    X_train, X_test, y_train, y_test = train_test_split(X, yilaç, test_size = 0.30, random_state = 42)
    lgbmmm.fit(X_train, y_train)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: 17-AAG: object, AZD-0530: object, AZD6244: object, Crizotinib: object, Erlotinib: object, Lapatinib: object, NVP-TAE684: object, Nilotinib: object, Nutlin-3a: object, PD-0325901: object, PD-0332991: object, PHA-665752: object, PLX4720: object, Paclitaxel: object, Sorafenib: object